In [2]:
%load_ext autoreload
%autoreload 2
%matplotlib widget

In [3]:
import sys
sys.path.append('../')

In [4]:
import numpy as np
from scipy.spatial import KDTree
from matplotlib import pyplot as plt
import tiledb

In [5]:
from pyspatialkit.dataobjects.geopointcloud import GeoPointCloud
from pyspatialkit.storage.pointcloud.geopointcloudlayer import GeoPointCloudLayer
from pyspatialkit.storage.pointcloud.tiledbsparsebackend import AXIS_NAMES, AXIS_NAMES_SET
from pyspatialkit.crs.geocrs import GeoCrs, NoneCRS
from pyspatialkit.processing.image.interpolation import interpolate_along_axis
from pyspatialkit.spacedescriptors.geobox3d import GeoBox3d

Jupyter environment detected. Enabling Open3D WebVisualizer.
[Open3D INFO] WebRTC GUI backend enabled.
[Open3D INFO] WebRTCWindowSystem: HTTP handshake server disabled.
TYPECHECKINGFalse


In [6]:
from tests.utils import get_tmp_path

In [7]:
pc = GeoPointCloud.from_xyz_file('../testdata/dom/dom2_32_734_5748_2_st.xyz', GeoCrs.from_epsg(25832))

In [8]:
pc.bounds

array([7.340010e+05, 5.748001e+06, 5.392000e+01, 7.359990e+05,
       5.749999e+06, 1.191000e+02])

In [9]:
raster = pc.to_georaster(pixel_size=1, interpolate_holes=True)

In [10]:
dir_path = get_tmp_path() / 'geopointlcoudlayer'
crs = GeoCrs.from_epsg(25832)
bounds = [0, 0, 0, 100.0, 100, 100]

In [ ]:
pclayer.delete_permanently()

In [12]:
pclayer = GeoPointCloudLayer(directory_path=dir_path, crs=crs, bounds=pc.bounds,
                             data_scheme=pc.data_scheme, build_pyramid=False, point_density=1)

extent:[1998.   1998.     65.18]


In [11]:
pclayer.write_data(pc)

In [13]:
requ = GeoBox3d.from_bounds(pc.bounds, crs=pc.crs)

In [14]:
res = pclayer.get_data_for_geobox3d(requ)

In [15]:
res.shape

(999999, 4)